In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq


data_files = {
    'train': 'data/DuReaderQG/train.json',
    'validation': 'data/DuReaderQG/dev.json'
}
raw_datasets = load_dataset("json", data_files=data_files)
print(raw_datasets)
print(raw_datasets['train'].features)

/Users/zhouke/Documents/project/stage1/stage1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question', 'id'],
        num_rows: 14520
    })
    validation: Dataset({
        features: ['context', 'answer', 'question', 'id'],
        num_rows: 984
    })
})
{'context': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'id': Value(dtype='int64', id=None)}


In [2]:
from transformers import AutoModelForSeq2SeqLM

checkpoint = 'langboat/mengzi-t5-base'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
import torch
def tokenize_function(example):
    result = tokenizer(
        example['question'],
        example['context'],
        truncation=True,
        padding=True,
        return_tensors='pt'
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['answer'],
            truncation=True,
            padding=True,
            return_tensors='pt'
        )['input_ids']
    #     result['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
    #     end_token_idx = torch.where(labels == tokenizer.eos_token_id)[1]
    #     for idx, end_idx in enumerate(end_token_idx):
    #         labels[idx][end_idx + 1:] = -100
        example['labels'] = labels
    # example['labels'] = example['answer']
    return result

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/14520 [00:00<?, ? examples/s]/Users/zhouke/Documents/project/stage1/stage1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 984/984 [00:00<00:00, 5686.58 examples/s]


In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model, padding=True)

In [12]:
from transformers import TrainingArguments

# training_args = TrainingArguments('output/model', per_device_train_batch_size=4, per_device_eval_batch_size=4, eval_strategy='epoch')
training_args = TrainingArguments(
    'output/model',
    per_device_train_batch_size=2,   # 调小 batch size
    per_device_eval_batch_size=2,    # 调小 eval batch size
    eval_strategy='epoch'
)
# training_args = TrainingArguments('output/model', eval_strategy='epoch')

In [13]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [14]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 